In [8]:
import pandas as pd

df = pd.read_csv('enronClean.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'enronClean.csv'

In [50]:
df.head()

,TID,MID,SUBJECT,FROM,TIMESTAMP,TO,TYPE
0,e94a22508dac953,e94a22508dac953,"""FW: LINE SM-123""",victor.lamadrid@enron.com,2001-10-01T14:19:03-07:00,john.hodge@enron.com,TO
1,e94a22508dac953,e94a22508dac953,"""FW: LINE SM-123""",victor.lamadrid@enron.com,2001-10-01T14:19:03-07:00,john.singer@enron.com,TO
2,e94a22508dac953,e94a22508dac953,"""FW: LINE SM-123""",victor.lamadrid@enron.com,2001-10-01T14:19:03-07:00,scott.neal@enron.com,TO
3,e94a22508dac953,e94a22508dac953,"""FW: LINE SM-123""",victor.lamadrid@enron.com,2001-10-01T14:19:03-07:00,clarissa.garcia@enron.com,TO
4,e94a22508dac953,e94a22508dac953,"""FW: LINE SM-123""",victor.lamadrid@enron.com,2001-10-01T14:19:03-07:00,chris.germany@enron.com,TO


In [56]:
df['FROM'].to_csv('from_column.csv', index=False)
df = pd.read_csv('from_column.csv')
unique_df = df.drop_duplicates()
unique_df.to_csv('email_ids.csv',index=False)

In [57]:
unique_df.head()


,FROM
0,victor.lamadrid@enron.com
11,mary.cook@enron.com
12,sara.shackleton@enron.com
13,michelle.nelson@enron.com
14,total@shagmail.com


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

def extract_username(email):
    return email.split('@')[0]

# Extract usernames from emails
usernames = df['email'].apply(extract_username)

# Generate n-grams (let's use unigrams and bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 2))  # n-gram range (1, 2) gives unigrams and bigrams
X_ngrams = vectorizer.fit_transform(usernames)
import spacy

# Load the English model from spaCy
nlp = spacy.load("en_core_web_sm")

def contains_person_name(username):
    doc = nlp(username)
    # Check if any entity in the username is a person
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            return 1
    return 0

# Apply NER to check if usernames contain a person’s name
df['has_person_name'] = df['email'].apply(lambda x: contains_person_name(extract_username(x)))

# Print to verify
print(df[['email', 'has_person_name']])
# Extract basic features as before
features = np.array([extract_features(email) for email in df['email']])

# Combine with NER features
features_with_ner = np.hstack((features, df['has_person_name'].values.reshape(-1, 1)))

# Now, you can add the n-grams features
X_combined = np.hstack((features_with_ner, X_ngrams.toarray()))

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


In [9]:
#seperating the emil ids from the prod data
import pandas as pd
file_path = '/Users/agoyal/Downloads/search-results-2024-09-25T23_01_59.518-0700.csv'

df = pd.read_csv(file_path)

user_ids = df[['message.message.user_id']]

user_ids.to_csv('user_ids.csv', index=False)

In [10]:
#filter out the empty strings
input_file_path = 'user_ids.csv'
output_file_path = 'emails_filtered.csv'

with open(input_file_path, 'r') as file:
    lines = file.readlines()

# Filter out empty strings and strip whitespace
emails = [line.strip() for line in lines if line.strip() != '""' and line.strip() != '']

# Write the valid emails to the output CSV file
with open(output_file_path, 'w') as file:
    for email in emails:
        file.write(email + '\n')

print(f"Valid emails have been saved to {output_file_path}")

Valid emails have been saved to emails_filtered.csv
